In [1]:
# De ser necesario
!pip install dash dash-bootstrap-components pandas plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 10.4 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

# Cargar los datos
results_dir = "/content/drive/My Drive/TFM/resultados"
results_df = pd.read_csv(os.path.join(results_dir, "resultados_modelo.csv"))
results_df_all_models = pd.read_csv(os.path.join(results_dir, "resultados_todos_modelos.csv"))
all_predictions_df = pd.read_csv(os.path.join(results_dir, "todas_las_predicciones.csv"))

# Crear una lista de modelos (extraerlos del DataFrame de predicciones)
model_columns = [col for col in all_predictions_df.columns if '_Pred' in col]

# Crear la lista de modelos con nombres limpios
model_list = [model.replace('_Pred', '') for model in model_columns]

# Inicializar la aplicación Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Definir el layout de la aplicación con la misma distribución que el dashboard anterior
app.layout = dbc.Container([
    html.H1("Dashboard de Resultados de Modelos", style={'textAlign': 'center'}),

    # Legend interactivo para la selección de modelos
    dbc.Row([
        dbc.Col(html.Label("Selecciona Modelos:"), width=12),
        dbc.Col(dcc.Checklist(
            id='model-checklist',
            options=[{'label': model, 'value': model} for model in model_list],
            value=model_list,  # Selecciona todos los modelos por defecto
            inline=True  # Aparecerán como una lista horizontal (legend)
        ), width=12)
    ], className="mb-4"),

    # Gráfico de RMSE y R² en la misma fila
    dbc.Row([
        dbc.Col(dcc.Graph(id='rmse-chart'), width=6),
        dbc.Col(dcc.Graph(id='r2-chart'), width=6),
    ]),

    # Tabla de RMSE y R² en el centro, reducida a la mitad
    dbc.Row([
        dbc.Col(html.H3("Tabla de RMSE y R²"), width={"size": 6, "offset": 3}),
        dbc.Col(html.Div(id='rmse-r2-table', style={'textAlign': 'center', 'width': '50%'}), width=12)  # Tamaño de la tabla reducido a la mitad
    ]),

    # Gráfico de predicciones de todos los modelos vs valores reales y mejor modelo en la misma fila
    dbc.Row([
        dbc.Col(dcc.Graph(id='all-predictions-chart'), width=6),
        dbc.Col(dcc.Graph(id='best-model-chart'), width=6),
    ]),
], fluid=True)

# Actualizar el gráfico de RMSE y R² basados en los modelos seleccionados
@app.callback(
    [Output('rmse-chart', 'figure'), Output('r2-chart', 'figure')],
    Input('model-checklist', 'value')
)
def update_rmse_r2_charts(selected_models):
    # Filtrar por los modelos seleccionados
    filtered_df = results_df_all_models[results_df_all_models['Modelo'].isin(selected_models)]

    # Gráfico de RMSE
    rmse_fig = px.bar(filtered_df, x='RMSE', y='Modelo', orientation='h', title="RMSE de Modelos Seleccionados",
                      labels={"RMSE": "RMSE", "Modelo": "Modelos"}, color='Modelo')

    # Gráfico de R²
    r2_fig = px.bar(filtered_df, x='R^2', y='Modelo', orientation='h', title="R² de Modelos Seleccionados",
                    labels={"R^2": "R²", "Modelo": "Modelos"}, color='Modelo')

    return rmse_fig, r2_fig

# Actualizar la tabla de RMSE y R²
@app.callback(
    Output('rmse-r2-table', 'children'),
    Input('model-checklist', 'value')
)
def update_rmse_r2_table(selected_models):
    filtered_df = results_df_all_models[results_df_all_models['Modelo'].isin(selected_models)]
    table = dbc.Table.from_dataframe(filtered_df[['Modelo', 'RMSE', 'R^2']], striped=True, bordered=True, hover=True)
    return table

# Actualizar el gráfico de todas las predicciones vs valores reales
@app.callback(
    Output('all-predictions-chart', 'figure'),
    Input('model-checklist', 'value')
)
def update_all_predictions_chart(selected_models):
    # Convertir el DataFrame de predicciones a formato largo con melt()
    all_predictions_df_long = all_predictions_df.melt(id_vars=['Valores Reales'],
                                                      value_vars=[model + '_Pred' for model in selected_models],
                                                      var_name='Modelo', value_name='Predicciones')

    # Limpiar los nombres de los modelos
    all_predictions_df_long['Modelo'] = all_predictions_df_long['Modelo'].str.replace('_Pred', '')

    # Crear gráfico
    fig = px.scatter(all_predictions_df_long, x='Valores Reales', y='Predicciones', color='Modelo',
                     title=f"Comparación de Predicciones vs Valores Reales para Modelos Seleccionados")
    fig.add_shape(type='line', line_color='red', line_width=2, x0=40, x1=90, y0=40, y1=90)  # Línea de identidad
    return fig

# Actualizar el gráfico del mejor modelo
@app.callback(
    Output('best-model-chart', 'figure'),
    Input('model-checklist', 'value')
)
def update_best_model_chart(selected_models):
    best_model_name = results_df_all_models.loc[results_df_all_models['RMSE'].idxmin(), 'Modelo']
    best_model_data = results_df  # Asegurar que se obtiene el dataframe con los datos

    if best_model_name in selected_models:
        fig = px.scatter(best_model_data, x='Valores Reales', y='Predicciones', title=f"Predicciones vs. Valores Reales ({best_model_name}) - Mejor Modelo")
        fig.add_shape(type='line', line_color='red', line_width=2, x0=40, x1=90, y0=40, y1=90)  # Línea de identidad
    else:
        fig = px.scatter([], x='Valores Reales', y='Predicciones', title=f"Mejor Modelo no Seleccionado")
    return fig

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>